In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('./drive/My Drive')

In [0]:
baseDir = '/content/drive/My Drive/Specs/'
trainDir = baseDir + 'train'
testDir = baseDir + 'test'
valDir = baseDir + 'validation'

In [0]:
width = 225
height = 150
epochs = 20
batch_size = 5
num_classes = 8

In [14]:
import keras
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense, Dropout

Using TensorFlow backend.


In [15]:
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(225, 150, 3))
conv_base.trainable = False  #freezing all layers of the vgg16 model

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
#dense layers
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(1024, activation='relu',input_dim=7*4*512))
model.add(Dropout(0.2))
model.add(layers.Dense(8, activation='softmax'))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              14681088  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 8200      
Total params: 14,689,288
Trainable params: 14,689,288
Non-trainable params: 0
_________________________________________________________________


In [0]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 225, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 225, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 225, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 75, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 75, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 75, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 37, 128)       0         
__________

In [0]:
#Extracting the features and labels as numpy arrays
#predict function of conv_base is used to get the features of the image
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 5
def extract_features(directory, sample_count):
  features = np.zeros(shape=(sample_count, 7, 4, 512))
  labels = np.zeros(shape=(sample_count,8))
  generator = datagen.flow_from_directory( directory,
  target_size=(225,150),
  batch_size=batch_size,
  shuffle = False,                               
  class_mode='categorical')
  print(generator.class_indices)
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = conv_base.predict(inputs_batch)
    features[i * batch_size : (i + 1) * batch_size] = features_batch
    labels[i * batch_size : (i + 1) * batch_size,:] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
        break
  return features, labels

In [19]:
train_features, train_labels = extract_features(trainDir, 1095)
validation_features, validation_labels = extract_features(valDir, 225)
test_features, test_labels = extract_features(testDir, 245)

Found 1095 images belonging to 8 classes.
{'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}
Found 225 images belonging to 8 classes.
{'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}
Found 245 images belonging to 8 classes.
{'Arijit Singh': 0, 'Armaan Malik': 1, 'Atif Aslam': 2, 'Honey Singh': 3, 'Raahat Fateh Ali Khan': 4, 'Shreya Ghoshal': 5, 'Sonu Nigam': 6, 'Sunidhi Chauhan': 7}


In [0]:
train_features = np.reshape(train_features, (1095, 7*4*512))
validation_features = np.reshape(validation_features, (225, 7*4*512))
test_features = np.reshape(test_features, (245, 7*4*512))

In [0]:
#os.getcwd()

'/content/drive/My Drive'

In [0]:
#np.savetxt('train_features.csv', train_features, fmt='1.8%f', delimiter = ',')

In [0]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr = 1e-4),metrics=['acc'])

In [22]:
history = model.fit(train_features, train_labels,
epochs=100,
batch_size=5,
validation_data=(validation_features, validation_labels))

Train on 1095 samples, validate on 225 samples
Epoch 1/100
1095/1095 [==============================] - 8s 7ms/step - loss: 2.4185 - acc: 0.1370 - val_loss: 2.1624 - val_acc: 0.1600
Epoch 2/100
1095/1095 [==============================] - 7s 6ms/step - loss: 2.0352 - acc: 0.2274 - val_loss: 2.0716 - val_acc: 0.1378
Epoch 3/100
1095/1095 [==============================] - 7s 6ms/step - loss: 1.9246 - acc: 0.2630 - val_loss: 2.0128 - val_acc: 0.2578
Epoch 4/100
1095/1095 [==============================] - 7s 6ms/step - loss: 1.8358 - acc: 0.3105 - val_loss: 1.9864 - val_acc: 0.2578
Epoch 5/100
1095/1095 [==============================] - 7s 6ms/step - loss: 1.7705 - acc: 0.3233 - val_loss: 1.9118 - val_acc: 0.2978
Epoch 6/100
1095/1095 [==============================] - 7s 6ms/step - loss: 1.6972 - acc: 0.3589 - val_loss: 1.8894 - val_acc: 0.3244
Epoch 7/100
1095/1095 [==============================] - 7s 6ms/step - loss: 1.6467 - acc: 0.3680 - val_loss: 1.8054 - val_acc: 0.3378
Epoch 8/

In [25]:
model.evaluate(test_features,test_labels, batch_size = 5)

245/245 [==============================] - 0s 640us/step


[2.5707582619269282, 0.4000000071768858]

In [0]:
model.save('voice_rec.h5')

In [0]:
from keras.models import load_model
model = load_model('voice_rec.h5')

In [0]:
test_predictions = model.predict(test_features,batch_size=5)

In [29]:

test_predictions.shape

(245, 8)

In [0]:
y_pred = []
for i in test_predictions:
  max = -1
  pos=-1
  for j in range(8):
    if i[j] > max:
      max=i[j]
      pos = j
  y_pred.append(pos)

In [0]:
y_test = []
for i in test_labels:
  max = -1
  pos=-1
  for j in range(8):
    if i[j] > max:
      max=i[j]
      pos = j
  y_test.append(pos)

In [0]:
y_train = []
for i in train_labels:
  max = -1
  pos=-1
  for j in range(8):
    if i[j] > max:
      max=i[j]
      pos = j
  y_train.append(pos)

In [0]:
y_pred = np.array(y_pred)
y_test = np.array(y_test)
y_train = np.array(y_train)

In [0]:
from sklearn.metrics import confusion_matrix

In [35]:
print(confusion_matrix(y_test,y_pred))

[[14  3  3  1  1  2  4  2]
 [ 5 18  5  0  0  1  0  1]
 [ 2  3 18  0  0  0  1  1]
 [ 6  1  0 15  1  0  2  5]
 [ 6  1 10  0 11  1  3  3]
 [ 2  3  2  0  0 16  3  4]
 [ 8  4  1  0  8  0  7  7]
 [ 4  3  0  1  1  6  5 10]]


In [0]:
singer_labels = { 0:'Arijit Singh',  1:'Armaan Malik',  2:'Atif Aslam' , 3:'Honey Singh', 4:'Raahat Fateh Ali Khan', 5:'Shreya Ghoshal', 6:'Sonu Nigam', 7:'Sunidhi Chauhan'}

In [0]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [0]:
clf = XGBClassifier(n_threads=-1,n_estimators=100)
clf.fit(train_features,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, n_threads=-1, nthread=None, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [0]:
clf.score(test_features,y_test)

0.45714285714285713

In [0]:
import joblib
#save model
joblib.dump(clf, 'XGB model') 



In [0]:
#load model
clf = joblib.load('XGB model')

In [39]:
y_pred_xg = clf.predict(test_features)
y_pred_xg

array([0, 4, 6, 1, 5, 0, 2, 0, 7, 0, 0, 6, 2, 5, 0, 0, 5, 2, 0, 0, 0, 1,
       7, 0, 7, 1, 2, 4, 4, 1, 1, 1, 6, 1, 6, 0, 1, 2, 1, 1, 1, 1, 0, 5,
       6, 5, 2, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 0, 1, 1, 5, 2, 0, 4, 5,
       1, 1, 1, 0, 1, 2, 1, 2, 2, 4, 1, 4, 2, 0, 0, 2, 1, 2, 2, 3, 3, 5,
       7, 4, 7, 5, 3, 3, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3,
       3, 3, 0, 0, 0, 5, 4, 4, 0, 0, 5, 0, 1, 4, 0, 4, 7, 0, 6, 6, 6, 4,
       2, 5, 4, 4, 5, 7, 4, 6, 6, 6, 4, 6, 7, 7, 4, 6, 4, 5, 5, 5, 5, 5,
       5, 5, 7, 2, 5, 5, 5, 5, 1, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, 5, 5,
       6, 5, 5, 5, 7, 0, 4, 6, 7, 4, 1, 4, 6, 6, 7, 0, 7, 6, 7, 1, 4, 6,
       0, 1, 1, 1, 0, 1, 6, 6, 6, 6, 6, 2, 0, 2, 1, 4, 4, 0, 7, 2, 1, 1,
       7, 4, 6, 1, 7, 5, 6, 7, 0, 4, 4, 0, 4, 5, 6, 7, 6, 7, 5, 7, 7, 4,
       7, 7, 7])

In [40]:
y_pred

array([0, 6, 0, 1, 5, 0, 2, 0, 0, 0, 0, 6, 6, 5, 0, 0, 0, 0, 1, 0, 2, 0,
       7, 0, 1, 7, 2, 3, 6, 4, 1, 1, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 0, 1,
       2, 5, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 7, 0, 2, 2, 6, 2, 2, 7,
       1, 2, 1, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 0, 1,
       0, 6, 0, 6, 7, 7, 7, 7, 3, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 0, 0, 0, 4, 7, 4, 4, 0, 0, 5, 0, 1, 0, 0, 4, 2, 0, 2, 2, 4, 4,
       2, 2, 4, 4, 2, 7, 2, 4, 6, 6, 4, 2, 4, 7, 2, 6, 4, 2, 5, 5, 6, 7,
       6, 7, 7, 5, 5, 5, 0, 5, 1, 5, 1, 1, 2, 5, 5, 5, 5, 5, 7, 5, 0, 6,
       2, 5, 5, 5, 7, 4, 0, 4, 4, 0, 0, 7, 6, 7, 7, 6, 7, 6, 7, 2, 4, 4,
       4, 0, 0, 1, 6, 1, 4, 1, 6, 6, 6, 7, 0, 0, 4, 1, 0, 6, 6, 6, 1, 1,
       5, 5, 5, 0, 7, 1, 6, 7, 0, 7, 0, 0, 4, 7, 6, 7, 3, 7, 5, 7, 7, 5,
       7, 5, 7])

In [41]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7])

In [0]:
from collections import Counter

In [53]:
ctr=0
correct = 0
count = 0
prev_singer = 0
for i in range(49):
  count_pred = Counter(y_pred[ctr:ctr+5]).most_common(2)

  actual_singer,num = Counter(y_test[ctr:ctr+5]).most_common(1)[0]
  if(actual_singer!=prev_singer):
    prev_singer = actual_
    count = 0
  
  #print(count_pred)
  pred_singers = []
  for pred in count_pred:
    pred_singers.append([singer_labels[pred[0]],pred[1]/5])
  #print(pred_singers)
  print('Song:',test_songs[singer_labels[actual_singer]][count])
  count+=1
  print('Actual Singer:', singer_labels[actual_singer])
  print('Predictions:-')
  max_conf_singer = 'None'
  tie = 0
  maxi = 0
  for singer in pred_singers:
      
      print('Singer:', singer[0])
      print('Confidence:', singer[1])
      if(singer[1]>maxi):
        if singer[1]>0.2:
          maxi = singer[1]
          max_conf_singer = singer[0]
      if(singer[1]==0.4):
        tie+=1
      
  #print(max_conf_singer)
  if(max_conf_singer!='None'):
    if max_conf_singer == singer_labels[actual_singer] and tie <= 1:
      correct+=1
      
  
  ctr+=5
  print()
print('Correct majority vote songs:', correct)

Song: Lambiyaan Si Judaiyaan Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.4
Singer: Sonu Nigam
Confidence: 0.2

Song: Nashe Si Chadh Gayi Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.8
Singer: Atif Aslam
Confidence: 0.2

Song: Enna Sona Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.4
Singer: Sonu Nigam
Confidence: 0.4

Song: Phir Le Aya Dil (Reprise) Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.8
Singer: Armaan Malik
Confidence: 0.2

Song: Baaton Ko Teri Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.4
Singer: Atif Aslam
Confidence: 0.2

Song: Yeh Fitoor Mera Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Sunidhi Chauhan
Confidence: 0.2
Singer: Atif Aslam
Confidence: 0.2

Song: Kaun Tujhe & Kuch Toh Hai Mashup Song 
Actual Singer: Armaan Malik
Predictions:-
Singer: Armaan Malik
Confidence: 0.6

In [51]:
ctr=0
correct = 0
count = 0
prev_singer = 0
for i in range(49):
  count_pred = Counter(y_pred_xg[ctr:ctr+5]).most_common(2)

  actual_singer,num = Counter(y_test[ctr:ctr+5]).most_common(1)[0]
  if(actual_singer!=prev_singer):
    prev_singer = actual_singer
    count = 0
  
  #print(count_pred)
  pred_singers = []
  for pred in count_pred:
    pred_singers.append([singer_labels[pred[0]],pred[1]/5])
  #print(pred_singers)
  print('Song:',test_songs[singer_labels[actual_singer]][count])
  count+=1
  print('Actual Singer:', singer_labels[actual_singer])
  print('Predictions:-')
  max_conf_singer = 'None'
  tie = 0
  maxi = 0
  for singer in pred_singers:
      
      print('Singer:', singer[0])
      print('Confidence:', singer[1])
      if(singer[1]>maxi):
        if singer[1]>0.2:
          maxi = singer[1]
          max_conf_singer = singer[0]
      if(singer[1]==0.4):
        tie+=1
      
  #print(max_conf_singer)
  if(max_conf_singer!='None'):
    if max_conf_singer == singer_labels[actual_singer] and tie <= 1:
      correct+=1
      
  
  ctr+=5
  print()
print('Correct majority vote songs:', correct)
print('Accuracy:', correct/49)

Song: Lambiyaan Si Judaiyaan Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.2
Singer: Raahat Fateh Ali Khan
Confidence: 0.2

Song: Nashe Si Chadh Gayi Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.6
Singer: Atif Aslam
Confidence: 0.2

Song: Enna Sona Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.4
Singer: Sonu Nigam
Confidence: 0.2

Song: Phir Le Aya Dil (Reprise) Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.6
Singer: Shreya Ghoshal
Confidence: 0.2

Song: Baaton Ko Teri Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Arijit Singh
Confidence: 0.4
Singer: Sunidhi Chauhan
Confidence: 0.4

Song: Yeh Fitoor Mera Song 
Actual Singer: Arijit Singh
Predictions:-
Singer: Armaan Malik
Confidence: 0.4
Singer: Raahat Fateh Ali Khan
Confidence: 0.4

Song: Kaun Tujhe & Kuch Toh Hai Mashup Song 
Actual Singer: Armaan Malik
Predictions:-
Singer: Ar

In [5]:
dirr = os.getcwd() + '/Songs_Dataset/test/'
dirr

'/content/drive/My Drive/Songs_Dataset/test/'

In [0]:
folders = os.listdir(dirr)

In [48]:
test_songs = {}
for folder in folders:
  songs = os.listdir(dirr+folder)
  #print(folder)
  for i in range(len(songs)):
    songs[i] = songs[i].split('-')[0]
  test_songs[folder] = songs
  #print(songs)
test_songs

{'Arijit Singh': ['Lambiyaan Si Judaiyaan Song ',
  'Nashe Si Chadh Gayi Song ',
  'Enna Sona Song ',
  'Phir Le Aya Dil (Reprise) Song ',
  'Baaton Ko Teri Song ',
  'Yeh Fitoor Mera Song '],
 'Armaan Malik': ['Kaun Tujhe & Kuch Toh Hai Mashup Song ',
  'Jaadugari Song ',
  'Khwaishein (Film Version) Song ',
  'Main Hoon Hero Tera (Sad Version) Song ',
  'Tum Jo Mile Song ',
  'Rendu Kallu Song ',
  'Sariyaagi (From '],
 'Atif Aslam': ['Mori Araj Suno (From ',
  'Jal Pari Song ',
  'Khair Mangda Song ',
  'Dil Kare Song ',
  'Darasal Song '],
 'Honey Singh': ['One Bottle Down Song ',
  'Khalaara Song ',
  'Superman Song ',
  'Rani Song ',
  'Love Dose Song ',
  'Urvashi Song '],
 'Raahat Fateh Ali Khan': ['Naino Nay Tere Song ',
  'Tum Kon Piya Song ',
  'Tinka Tinka Dil Mera Song ',
  'Saiyaan Radio ',
  'Fakeera Song ',
  'Lag Ja Gale Song ',
  'Tera Mera Sath Ho Song '],
 'Shreya Ghoshal': ['Sunn Raha Hai (Female) Song ',
  'Yeh Kya Hua Song ',
  'Agar Tum Mil Jao',
  'Sunn Raha Ha